In [2]:
import numpy as np
import pandas as pd
import random
from sklearn.metrics import classification_report

In [3]:
df = pd.read_csv(r"C:\Users\varun george\OneDrive\Desktop\SNU\4th sem\mlt\datasets\Bank_Personal_Loan_Modelling.csv")

In [4]:
df.drop(["ID", "ZIP Code"],axis=1,inplace=True)

In [5]:
df["Experience"] = abs(df["Experience"])

In [6]:
df.head()

,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,25,1,49,4,1.6,1,0,0,1,0,0,0
1,45,19,34,3,1.5,1,0,0,1,0,0,0
2,39,15,11,1,1.0,1,0,0,0,0,0,0
3,35,9,100,1,2.7,2,0,0,0,0,0,0
4,35,8,45,4,1.0,2,0,0,0,0,0,1


In [8]:
df['target'] = df['Personal Loan']
df.drop('Personal Loan',axis=1,inplace=True)

In [51]:
x = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [52]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=69)

In [53]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)
x_train.shape, y_train.shape

((3750, 11), (3750,))

### MODEL - USING PYTORCH

In [54]:
import torch
from torch.utils.data import DataLoader, TensorDataset

train_x = torch.from_numpy(x_train).to(torch.float32)
train_y = torch.from_numpy(y_train).to(torch.float32)
train_x.shape, train_y.shape



(torch.Size([3750, 11]), torch.Size([3750]))

In [55]:
(torch.Size([3750, 11]), torch.Size([3750]))
data = TensorDataset(train_x,train_y)
data = DataLoader(data,batch_size=20,shuffle=True)

In [56]:
class Model(torch.nn.Module):
    
    def __init__(self):
        super(Model,self).__init__()
        
        self.layer1 = torch.nn.Linear(11,4)
        self.layer2 = torch.nn.Linear(4,1)
        self.sigmoid = torch.nn.Sigmoid()
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.sigmoid(x)
        return x

In [57]:
model=Model()
model

Model(
  (layer1): Linear(in_features=11, out_features=4, bias=True)
  (layer2): Linear(in_features=4, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (relu): ReLU()
)

### OPTIMAIZATION OF WEIGHTS

In [58]:
from pyswarms.single import GlobalBestPSO
torch.set_grad_enabled(False)

param = np.concatenate([i.numpy().flatten() for i in model.parameters()])
shape = [i.numpy().shape for i in model.parameters()]
size = [i[0]*i[1] if len(i) == 2 else i[0] for i in shape]


In [59]:
print("Dim : ", len(param))
print("Shape of different layers : ", shape)

Dim :  53
Shape of different layers :  [(4, 11), (4,), (1, 4), (1,)]


In [64]:
def objective_function(particle,shape=shape,size=size):
    acc_score = []
    output = []
    
    for par in particle:
        param=list()
        cum_sum = 0
        for i in range(len(size)):
            array = par[cum_sum : cum_sum + size[i]]
            array = array.reshape(shape[i])
            cum_sum += size[i]
            param.append(array)
        param = np.array(param, dtype="object")
        output.append(param)
    
    for i in range(len(output)):
        # Copy Weights and Biases
        model = Model()
        for idx,weights in enumerate(model.parameters()):
            weights.data = (torch.tensor(output[i][idx])).to(torch.float32)
        
        # Calculate Accuracy
        y_pred = model(train_x)
        y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
        acc = (y_pred == train_y).sum().float().item() / len(data.dataset)
        acc_score.append(1 - acc) # Optimization function aims to reduce the cost so (1 - accuracy)
        
    return acc_score

In [65]:
grid = {'c1': 0.6, 'c2': 0.3, 'w': 0.1}
dim = len(param)
x_max = 1.0 * np.ones(dim)
x_min = -1.0 * x_max
bounds = (x_min, x_max)

pso = GlobalBestPSO(n_particles=500, dimensions=53, options=grid, bounds=bounds)

In [66]:
best_cost, best_parameters = pso.optimize(objective_function, iters=20)
print("Accuracy : ", 1 - best_cost)

2023-04-05 23:05:50,497 - pyswarms.single.global_best - INFO - Optimize for 20 iters with {'c1': 0.6, 'c2': 0.3, 'w': 0.1}
pyswarms.single.global_best: 100%|█████████████████████████████████████████████████████████████|20/20, best_cost=0.0405
2023-04-05 23:05:58,455 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.04053333333333331, best pos: [-0.522994    0.34208217 -0.04398486 -0.73788285  0.16897916 -0.33793491
  0.08642112 -0.26926862 -0.32940727 -0.07397394 -0.04961181  0.12101332
 -0.02207978  0.54524444  0.04258377  0.32955925  0.30526291  0.07025747
 -0.19271258  0.1707581  -0.06437067 -0.21468247  0.22043951 -0.07545865
  0.18868823  0.11819435 -0.31602114  0.52468413 -0.04465821  0.30739816
 -0.11058351 -0.13512382 -0.30168188  0.04561092  0.40277969 -0.34832955
  0.32548779 -0.25622949  0.18499545  0.18975475 -0.03178957 -0.15068021
  0.20801496 -0.14960711  0.15698655 -0.3572919  -0.01562668 -0.16613613
 -0.30760811  0.33100646  0.02668062 -0.1507

Accuracy :  0.9594666666666667


In [67]:
result = []
for par in [best_parameters]:
    param = list()
    cum_sum = 0
    for i in range(len(size)):
        array = par[cum_sum : cum_sum + size[i]]
        array = array.reshape(shape[i])
        cum_sum += size[i]
        param.append(array)
    param = np.array(param, dtype="object")
    result.append(param)

best_model = Model()
for idx,wei in enumerate(best_model.parameters()):
    wei.data = (torch.tensor(result[0][idx])).to(torch.float32)

# Calculate Accuracy
y_pred = best_model(train_x)
y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
acc = (y_pred == train_y).sum().float().item() / len(data.dataset)
print("Accuracy : ", acc)

Accuracy :  0.9594666666666667


In [68]:
test_x = torch.from_numpy(x_test).to(torch.float32)
test_y = torch.from_numpy(y_test).to(torch.float32)
test = TensorDataset(test_x,test_y)
test = DataLoader(test,batch_size=1)

In [69]:
y_pred = best_model(test_x)
y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
print(classification_report(y_pred,test_y))

              precision    recall  f1-score   support

           0       0.99      0.97      0.98      1157
           1       0.66      0.82      0.73        93

    accuracy                           0.96      1250
   macro avg       0.82      0.89      0.85      1250
weighted avg       0.96      0.96      0.96      1250

